In [15]:
import pandas as pd
df = pd.read_csv("fraud_oracle.csv")

In [16]:
#removing bad age values
df = df[df['Age'] != 0]

#removing policy number and year
df = df.drop(["PolicyNumber","Year"],axis=1)

In [17]:
#setting a number value for each month and day of the week
month_to_number = {
    'January': 1, 'Jan': 1,
    'February': 2, 'Feb': 2,
    'March': 3, 'Mar': 3,
    'April': 4, 'Apr': 4,
    'May': 5,
    'June': 6, 'Jun': 6,
    'July': 7, 'Jul': 7,
    'August': 8, 'Aug': 8,
    'September': 9, 'Sep': 9,
    'October': 10, 'Oct': 10,
    'November': 11, 'Nov': 11,
    'December': 12, 'Dec': 12
}
day_to_number = {
    'Monday': 1, 'Mon': 1,
    'Tuesday': 2, 'Tue': 2,
    'Wednesday': 3, 'Wed': 3,
    'Thursday': 4, 'Thu': 4,
    'Friday': 5, 'Fri': 5,
    'Saturday': 6, 'Sat': 6,
    'Sunday': 7, 'Sun': 7
}

In [18]:
#changing the months and days values to numerical values
df['Month'] = df['Month'].map(month_to_number)
df['MonthClaimed'] = df['MonthClaimed'].map(month_to_number)
df['DayOfWeek'] = df['DayOfWeek'].map(day_to_number)
df['DayOfWeekClaimed'] = df['DayOfWeekClaimed'].map(day_to_number)

In [19]:
from sklearn.preprocessing import LabelEncoder

# List of column names to encode
columns_to_encode = ['AccidentArea','Sex','MaritalStatus','Fault','VehicleCategory','PastNumberOfClaims','PoliceReportFiled','WitnessPresent','AgentType','BasePolicy']

# LabelEncoder for each selected column
label_encoders = {}

# Encode the selected columns
for column in columns_to_encode:
    label_encoder = LabelEncoder()
    df[column] = label_encoder.fit_transform(df[column])
    label_encoders[column] = label_encoder

In [20]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# columns to be one-hot encoded
columns_to_encode = ['Make', 'PolicyType', 'VehiclePrice','Days_Policy_Accident','Days_Policy_Claim','AgeOfVehicle','AgeOfPolicyHolder','AddressChange_Claim','NumberOfCars','NumberOfSuppliments']

encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(df[['Make', 'PolicyType', 'VehiclePrice','Days_Policy_Accident','Days_Policy_Claim','AgeOfVehicle','AgeOfPolicyHolder','AddressChange_Claim','NumberOfCars','NumberOfSuppliments']])
df_encoded = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(['Make', 'PolicyType', 'VehiclePrice','Days_Policy_Accident','Days_Policy_Claim','AgeOfVehicle','AgeOfPolicyHolder','AddressChange_Claim','NumberOfCars','NumberOfSuppliments']))


In [21]:
df = df.drop(['Make', 'PolicyType', 'VehiclePrice','Days_Policy_Accident','Days_Policy_Claim','AgeOfVehicle','AgeOfPolicyHolder','AddressChange_Claim','NumberOfCars','NumberOfSuppliments'],axis=1)
df = df.reset_index()
df = df.drop('index',axis=1)
df_encoded = df_encoded.reset_index()
df_encoded = df_encoded.drop('index',axis=1)

#concat both data frames
concatenated_df = pd.concat([df, df_encoded], axis=1)